In [ ]:
import sys, os, re, gzip, json, pickle, shutil, random

import numpy as np

from scipy import stats

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from fse.models import SIF
from fse import IndexedList

from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data_path = '../data'